# TP6 - Procesamiento de Lenguaje Natural

Imports

In [1]:
import pandas as pd
import scipy.io
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns; sns.set_style("darkgrid")
import scipy
import itertools
import random
import math
import operator
import re
import xml.etree.ElementTree as ET
from lxml import etree
from collections import Counter
sns.plt = plt

## 1 - Asociacion de palabras
1.1 Levantar el corpus AP, separando cada noticia como un elemento distinto en un diccionario ( < doc_no > : < text > ).

Librerias necesarias: html5lib, lxml, bs4, re

In [2]:
ap_xml_data = open('data/ap.txt').read()

def xml2df(xml_data):
    root = ET.XML(xml_data, parser=etree.XMLParser(recover=True)) # element tree
    all_records = []
    for i, child in enumerate(root):
        record = {}
        for subchild in child:
            if subchild.tag=='DOCNO':
                record[subchild.tag] = subchild.text
            else:
                record[subchild.tag] = re.sub('[^a-zA-Z*]', ' ', subchild.text) 
            # Aquella regex elimina todo lo que no son letras.
            # Para aceptar numeros, puntos y guiones: [^a-zA-Z0-9-_*.]
            all_records.append(record)
    return pd.DataFrame(all_records).drop_duplicates().reset_index(drop=True)

ap_df = xml2df(ap_xml_data)

print "AP cargado."
print "Resumen de los primeros 10 documentos:"
print ap_df[0:10]

AP cargado.
Resumen de los primeros 10 documentos:
             DOCNO                                               TEXT
0   AP881218-0003     A    year old student at a private Baptist s...
1   AP880224-0195     The Bechtel Group Inc  offered in      to se...
2   AP881017-0144     A gunman took a    year old woman hostage af...
3   AP881017-0219     Today is Saturday  Oct      the    rd day of...
4   AP900117-0022     Cupid has a new message for lovers this Vale...
5   AP880405-0167     The Reagan administration is weighing whethe...
6   AP880825-0239     More than         skins of a protected speci...
7   AP880325-0232     There will be no organized union boost behin...
8   AP880908-0056     Here is a summary of developments in forest ...
9   AP881105-0097     Jean Pierre Stirbois  the No    man in the e...


1.2 Calcular el tamano del vocabulario.

In [3]:
freq_vocab = dict()
for index, row in ap_df.iterrows():
    for word in row[1].split():
        freq_vocab[word] = freq_vocab.get(word, 0) + 1

In [4]:
print "El vocabulario tiene: " + str(len(freq_vocab)) + " palabras."

El vocabulario tiene: 38520 palabras.


1.3 Para las 500 palabras con mas apariciones, calcular el par mas asociado segun la medida presentada.

In [5]:
max_freq_vocab = dict(sorted(freq_vocab.iteritems(), key=operator.itemgetter(1), reverse=True)[:500])
print "Se obtuvieron las " + str(len(max_freq_vocab)) + " palabras con mas apariciones."

Se obtuvieron las 500 palabras con mas apariciones.


Sean X,Y palabras, N la cantidad de palabras de todos los textos, W la ventana de co-ocurrencia:

f(X)=occurs(X). f(X,Y)=occurs(Y despues de X, a distancia <= W)/(W-1).

P(X)=f(X)/N. P(X,Y)=f(X,Y)/N.

I(X,Y)=log2(P(X,Y) / (P(X) x P(Y)))=log2( ( f(X,Y) x N ) / ( f(X) x f(Y) ))

In [12]:
def calculate_mutual_association(txts, relevant_words, W):
    N=0
    mutual_assoc = dict()
    for txt in txts:
        words = txt.split()
        N = N + len(words)
        for i in range(len(words)-W):
            word = words[i]
            if word in relevant_words:
                for j in range(1,W):
                    other = words[i+j]
                    mutual_assoc[word] = mutual_assoc.get(word, dict())
                    mutual_assoc[word][other] = mutual_assoc[word].get(other, 0.0) + 1.0
    for word, freqs in mutual_assoc.iteritems():
        for other, freq in freqs.iteritems():
            fxy = freq / (W-1)
            fx = freq_vocab[word]
            fy = freq_vocab[other]
            if fxy > 0:
                mutual_assoc[word][other] = math.log((fxy*N)/(fx*fy),2)
            else:
                mutual_assoc[word][other] = 0
    return mutual_assoc

In [13]:
mutual_assoc = calculate_mutual_association(ap_df.iloc[:,1], max_freq_vocab.keys(), 5)

In [15]:
for word, freqs in mutual_assoc.items():
    print "(" + word + "," + str(max(freqs.iteritems(), key=operator.itemgetter(1))[0]) + ")"

(all,tantalizing)
(dollar,depreciated)
(month,enjoined)
(four,Troiullot)
(go,immature)
(Communist,holdover)
(children,alcoholics)
(issues,Labronislav)
(whose,Mk)
(to,meld)
(program,Documentation)
(th,Breanne)
(under,Basins)
(must,sturdier)
(case,costliest)
(far,flung)
(condition,Tieman)
(service,Bikle)
(school,buddy)
(did,backflips)
(companies,Borden)
(German,Juden)
(small,bordered)
(George,Peppard)
(says,Blitz)
(leaders,restates)
(Rep,Fernand)
(past,Irrigation)
(Department,Standards)
(State,Shearn)
(cost,Cremation)
(air,brooch)
(even,Moses)
(index,propriatary)
(what,Abduleh)
(power,blackouts)
(while,Bookstore)
(spokesman,Satterfield)
(capital,Courtroom)
(new,Trivedi)
(America,Rafko)
(public,penetrating)
(told,Minot)
(led,Shizuka)
(men,Hosli)
(here,Hinojosa)
(hours,Ange)
(reported,Sunland)
(Ms,Implementation)
(vice,Leber)
(change,notwithstanding)
(Tuesday,Perus)
(reports,measurable)
(trial,litigators)
(action,Proposals)
(military,forcefully)
(family,mull)
(Robert,Felise)
(When,ADAPTED)

## 2 -  Informacion Lexica
Bajar de Project Gutenberg el libro de Darwin ON THE ORIGIN OF SPECIES.

2.1 Procesar el texto, tokenizando eliminando signos de puntuacion.

2.2 Siguiendo el artıculo de la seccion, calcular la autocorrelacion para estimar la distribucion de la palabra a lo largo del texto.

2.3 Armar una funcion que reciba una lista de tokens, una lista de palabras y un tamano de ventana y devuelva una lista de probabilidades de encontrar la palabra en cada ventana para cada palabra pasada por parametro.

2.4 Calcular la entropıa de la distribucion de palabras seleccionadas para distintos tamanos de ventana

2.5 Generar una version randomizada del texto, y medir la entropia de las palabras randomizadas.

2.6 Distinguir las palabras del texto en artıculos, sustantivos y adjetivos usando un POS-tagger. Verificar si las medidas separan a estos grupos de palabras.

## 3 - Word embeddings, distancia semantica y Word- Net
3.1 Utilizando el test WordSim3531, comparar el rendimiento entre LSA[3] y Word2Vec2 [4].

3.2 Comparar los distintos word embeddings con las medidas definidas en WordNet.